# Creacion del modelo 

In [1]:
# Importo las librerias correspondientes para la creacion del modelo

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import hstack, csr_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



### 1. Obtencion y organizacion de los datos

In [2]:
# Cargo el dataset modificado en el EDA

dataframe_unido_modelo = pd.read_parquet('../Datasets/dataset_unido_modelo')
print(f"Dimensiones del DataFrame: {dataframe_unido_modelo.shape}")

Dimensiones del DataFrame: (24004, 12)


In [3]:
# Combino las columnas relevantes en una sola columna para la vectorización

dataframe_unido_modelo['texto_combinado'] = (
    dataframe_unido_modelo['cast_name_actor'] + ' ' +
    dataframe_unido_modelo['crew_name_member'] + ' ' +
    dataframe_unido_modelo['overview']
)
# Reemplazo valores nulos en la columna combinada con una cadena vacía
dataframe_unido_modelo['texto_combinado'] = dataframe_unido_modelo['texto_combinado'].fillna('')

# Elimino caracteres no deseados como comas
dataframe_unido_modelo['texto_combinado'] = dataframe_unido_modelo['texto_combinado'].str.replace(',', ' ')

# guardo el archivo que va a ser usado en la api
dataframe_unido_modelo.to_parquet('../Datasets/dataset_unido_modelo', index=False)


### 2. Vectorizacion de las características numericas

In [4]:
# Vectorizo las características numéricas

caracteristicas_numericas = ['budget', 'revenue', 'vote_count', 'popularity']
escala = StandardScaler()
caracteristicas_numericas_normalizadas = escala.fit_transform(dataframe_unido_modelo[caracteristicas_numericas])
matrix_numerica_escalada = csr_matrix(caracteristicas_numericas_normalizadas)


### 3. Vectorización de las características categóricas

In [5]:
# Vectorizo el género y aplico un peso mayor a esta matriz

vectorizar_genero = TfidfVectorizer(stop_words='english')
matrix_tfidf_genero = vectorizar_genero.fit_transform(dataframe_unido_modelo['name_genre'])
peso_del_genero = 6.0
matrix_tfidf_genero_ponderado = peso_del_genero * matrix_tfidf_genero

# Vectorizo el texto combinado (actores, directores, overview)

vectorizar_texto_combinado = TfidfVectorizer(stop_words='english')
matrix_tfidf_combinado = vectorizar_texto_combinado.fit_transform(dataframe_unido_modelo['texto_combinado'])


### 4. Combinación de características:


In [6]:
# Combino la matriz numérica, la matriz ponderada de géneros y la matriz de texto combinado
caracteristicas_combinadas = hstack([matrix_numerica_escalada, matrix_tfidf_genero_ponderado, matrix_tfidf_combinado])

### 5. Realizar el calculo de la similitud del coseno

In [7]:
# Aseguro que la matriz combinada es de tipo csr_matrix
if not isinstance(caracteristicas_combinadas, csr_matrix):
    caracteristicas_combinadas = csr_matrix(caracteristicas_combinadas)

# Reduzco la dimensionalidad con SVD
svd = TruncatedSVD(n_components=100)
caracteristicas_reducidas = svd.fit_transform(caracteristicas_combinadas)

# Calculo la matriz de similitud del coseno
similitud_del_coseno = cosine_similarity(caracteristicas_reducidas)

In [8]:
# Función para obtener las mejores recomendaciones
# Preproceso los titulos en minusculas solo una vez fuera de la funcion
dataframe_unido_modelo['title_lower'] = dataframe_unido_modelo['title'].str.lower()
def recomendacion(title: str):
    """
    Recomienda películas similares a una película dada basada en la similitud del coseno.

    Parámetros:
        title: El título de la película para la cual se desean obtener recomendaciones.
    
    Retorna:
        Si el título es válido, retorna una lista de 5 títulos de películas recomendadas que son más similares.
        Si el título no es válido, retorna un mensaje indicando que el título no está disponible.
    """
    # Normalizo el título para comparar sin importar mayúsculas/minúsculas
    title = title.lower()

    # Verifico si el título está en el DataFrame
    if title not in dataframe_unido_modelo['title_lower'].values:
        return {"error": f"La película '{title}' no se encuentra dentro de la muestra de datos."}

    # Obtengo el índice de la película dada
    idx = dataframe_unido_modelo[dataframe_unido_modelo['title_lower'] == title].index[0]

    # Si la matriz de similitud es dispersa, trabajo directamente con ella sin convertir a densa
    if isinstance(similitud_del_coseno, csr_matrix):
        sim_scores = similitud_del_coseno[idx].toarray().flatten()
    else:
        sim_scores = similitud_del_coseno[idx]

    # Obtener las 5 tuplas más cercanas usando min-heap
    top_5_indices = np.argsort(sim_scores)[::-1]  # Ordena de mayor a menor
    top_5_indices = top_5_indices[sim_scores[top_5_indices] > 0]  # Filtra las similitudes positivas
    top_5_indices = [i for i in top_5_indices if i != idx][:5]  # Excluye la película misma y toma las 5 mejores

    # Obtener los títulos de las películas recomendadas
    top_5_titles = dataframe_unido_modelo.iloc[top_5_indices]['title'].tolist()

    return {"recomendaciones": top_5_titles}

In [9]:
# Realizo una verificacion con distintos titulos de peliculas para observar el funcionamiento del modelo

print("Recomendaciones para 'The Avengers':")
print(recomendacion('The Avengers'))

print("\nRecomendaciones para 'Men in Black':")
print(recomendacion('Men in Black'))

print("\nRecomendaciones para 'toy story':")
print(recomendacion('toy story'))

print("\nRecomendaciones para 'john wick:")
print(recomendacion('john wick'))

print("\nRecomendaciones para 'saw':")
print(recomendacion('saw'))

print(recomendacion('bee'))

Recomendaciones para 'The Avengers':
{'recomendaciones': ['Out of Time', 'The Missing', 'Hard Rain', 'U.S. Marshals', 'Broken City']}

Recomendaciones para 'Men in Black':
{'recomendaciones': ['The Incredibles', 'Terminator 2: Judgment Day', 'Sherlock Holmes', 'The Bourne Ultimatum', 'Captain America: The Winter Soldier']}

Recomendaciones para 'toy story':
{'recomendaciones': ['Despicable Me', 'Ice Age', 'Monsters, Inc.', 'Aladdin', 'Toy Story 2']}

Recomendaciones para 'john wick:
{'recomendaciones': ['Baby Driver', 'Wonder Woman', 'Minions', 'Deadpool', 'Gone Girl']}

Recomendaciones para 'saw':
{'recomendaciones': ['The Purge: Anarchy', '28 Days Later', 'Insidious', 'It Follows', 'Shaun of the Dead']}
{'error': "La película 'bee' no se encuentra dentro de la muestra de datos."}


### Para reducir la memoria en la api se tomo una version mas reducida del modelo anterior pero ambos funcionan con la misma cantidad de datos solo que en la api se crearon dos funciones y la similitud del coseno se creo dentro de la funcion de recomendaciones, se dejan las dos versiones con sus respectivas pruebas

In [10]:
# Vectorizo las características numéricas y género en funciones separadas para reducir el uso de memoria.
def procesar_datos(dataframe):
    """
    Procesa los datos del DataFrame para generar una matriz de características combinadas y reducidas.

    Este procesamiento incluye:
    1. Escalado de las características numéricas.
    2. Vectorización y ponderación del genero.
    3. Vectorización del texto combinado (actores, directores, overview).
    4. Combinación de todas las matrices vectorizadas en una sola.
    5. Reducción de dimensionalidad mediante SVD para obtener una representación mas compacta.

    Parametros:
    
        dataframe : el dataFrame 'dataframe_unido_modelo' que contiene las caracteristicas numericas, los generos y el texto combinado.

    Retorna:
    
        caracteristicas_reducidas : una matriz con las caracteristicas combinadas y reducidas dimensionalmente.
    """
    # Escalo las caracteristicas numericas
    caracteristicas_numericas = ['budget', 'revenue', 'vote_count', 'popularity']
    escala = StandardScaler()
    caracteristicas_numericas_normalizadas = escala.fit_transform(dataframe[caracteristicas_numericas])
    matrix_numerica_escalada = csr_matrix(caracteristicas_numericas_normalizadas)
    
    # Vectorizo y pondero el genero
    vectorizar_genero = TfidfVectorizer(stop_words='english')
    matrix_tfidf_genero = vectorizar_genero.fit_transform(dataframe['name_genre'])
    peso_del_genero = 6.0
    matrix_tfidf_genero_ponderado = peso_del_genero * matrix_tfidf_genero
    
    # Vectorizo el texto combinado (actores, directores, overview)
    vectorizar_texto_combinado = TfidfVectorizer(stop_words='english')
    matrix_tfidf_combinado = vectorizar_texto_combinado.fit_transform(dataframe['texto_combinado'])

    # Combino todas las matrices en una sola (caracteristicas numericas, genero y texto combinado)
    caracteristicas_combinadas = hstack([matrix_numerica_escalada, matrix_tfidf_genero_ponderado, matrix_tfidf_combinado])
    
    # Reduzco la dimensionalidad con SVD a 50 componentes para mantenerlo liviano
    svd = TruncatedSVD(n_components=50)
    caracteristicas_reducidas = svd.fit_transform(caracteristicas_combinadas)
    
    return caracteristicas_reducidas

# Proceso el DataFrame (solo se hace una vez)
dataframe_unido_modelo['title_lower'] = dataframe_unido_modelo['title'].str.lower()
caracteristicas_reducidas = procesar_datos(dataframe_unido_modelo)

# Creo una funcion de recomendacion con el cálculo de similitud del coseno dentro
def recomendaciones(title: str):
    """
    Recomienda películas similares a una película dada basada en la similitud del coseno.
    
    Parámetros:
        title: El título de la película para la cual se desean obtener recomendaciones.
        
    Retorna:
        Una lista de 5 títulos de películas recomendadas, o un mensaje de error si no se encuentra el título.
    """
    # Normalizo el título para comparar sin importar mayúsculas/minúsculas
    title = title.lower()

    # Verifico si el título está en el DataFrame
    if title not in dataframe_unido_modelo['title_lower'].values:
        return {"error": f"La película '{title}' no se encuentra en la base de datos."}

    # Obtengo el índice de la película dada
    idx = dataframe_unido_modelo[dataframe_unido_modelo['title_lower'] == title].index[0]

    # Calculo la matriz de similitud del coseno sobre la marcha para evitar mantenerla en memoria
    sim_scores = cosine_similarity(caracteristicas_reducidas[idx].reshape(1, -1), caracteristicas_reducidas).flatten()

    # Obtener las 5 películas más similares, excluyendo la misma película
    top_5_indices = np.argsort(sim_scores)[::-1][1:6]  # [1:6] excluye la película misma
    top_5_titles = dataframe_unido_modelo.iloc[top_5_indices]['title'].tolist()

    return {"recomendaciones": top_5_titles}


In [11]:
# Realizo una verificacion con distintos titulos de peliculas para observar el funcionamiento del modelo

print("Recomendaciones para 'The Avengers':")
print(recomendaciones('The Avengers'))

print("\nRecomendaciones para 'Men in Black':")
print(recomendaciones('Men in Black'))

print("\nRecomendaciones para 'toy story':")
print(recomendaciones('toy story'))

print("\nRecomendaciones para 'john wick:")
print(recomendaciones('john wick'))

print("\nRecomendaciones para 'saw':")
print(recomendaciones('saw'))

print(recomendaciones('bee'))

Recomendaciones para 'The Avengers':
{'recomendaciones': ['Out of Time', 'The Missing', 'Hard Rain', 'U.S. Marshals', 'Broken City']}

Recomendaciones para 'Men in Black':
{'recomendaciones': ['The Incredibles', 'Terminator 2: Judgment Day', 'Sherlock Holmes', 'The Bourne Ultimatum', 'Captain America: The Winter Soldier']}

Recomendaciones para 'toy story':
{'recomendaciones': ['Despicable Me', 'Ice Age', 'Monsters, Inc.', 'Aladdin', 'Toy Story 2']}

Recomendaciones para 'john wick:
{'recomendaciones': ['Baby Driver', 'Wonder Woman', 'Minions', 'Deadpool', 'Gone Girl']}

Recomendaciones para 'saw':
{'recomendaciones': ['The Purge: Anarchy', '28 Days Later', 'Insidious', 'It Follows', 'Shaun of the Dead']}
{'error': "La película 'bee' no se encuentra en la base de datos."}
